# TP3 - Problema 1

### Grupo 23

Pedro Gonçalves a101250
<br>
José Loureiro a96467
<br>
Bruno Neiva a95311

In [1]:
from pysmt.shortcuts import *
from pysmt.typing import INT
import itertools
bits = 9

### Modelação de predicados
O estado inicial é caracterizado pelo seguinte predicado:

$$ node = 0 \land a > 0 \land b > 0 $$

##### Nota: 
Estado atual = node <br> Estado seguinte = next 

As transições possíveis são caracterizadas pelo seguinte predicado:

$$ node = 0 \land next = 1 \land r = a \land r' = b \land s = 1 \land s' = 0 \land t = 0 \land t' = 1 $$

$$ \lor $$

$$ node = 1 \land next = 1 \land r' \neq 0 \land q = r // r' \land r = r' \land r' = r - (q * r') \land $$

$$ s = s' \land s' = s - (q * s') \land t = t' \land t' = t - (q * t') $$

$$ \lor $$

$$ node = 1 \land next = 3 \land (r = 0 \lor overflow) $$

$$ \lor $$

$$ node = 1 \land next = 2 \land r' = 0 \land r = r \land r' = r' \land s = s \land s' = s' \land t = t \land t' = t' $$



A função declare declara os estados e as variáveis que são utilizadas no problema.

In [2]:
def declare(i, n, s):
    state = {}
    
    # definir variáveis
    state['node'] = Symbol('node' + '!' + s + str(i), BVType(n))
    state['r1'] = Symbol('r1'+'!'+s + str(i), BVType(n))
    state['r2'] = Symbol('r2'+'!'+s + str(i), BVType(n))
    state['s1'] = Symbol('s1'+'!'+s + str(i), BVType(n))
    state['s2'] = Symbol('s2'+'!'+s + str(i), BVType(n))
    state['t1'] = Symbol('t1'+'!'+s + str(i), BVType(n))
    state['t2'] = Symbol('t2'+'!'+s + str(i), BVType(n))
    
    return state

A função init é responsável por inicializar o estado inicial.

In [3]:
def init(state, n):
    return And(
        Equals(state['node'], BV(0,n)),
        BVUGT(state['r1'], BV(0,n)),
        BVUGT(state['r2'], BV(0,n)),
        Equals(state['s1'], BV(1,n)),
        Equals(state['s2'], BV(0,n)),
        Equals(state['t1'], BV(0,n)),
        Equals(state['t2'], BV(1,n))
    )

A função trans é responsável por definir as transições possíveis entre estados.

In [4]:

def trans(atual, prox):
    
    overflow = 2**bits - 1
    m = BV(overflow, bits)
    
    # node = 0 ∧ next = 0 ∧ r' != 0 ∧ q = r // r' ∧ r = r' ∧ r' = r - (q * r') ∧ 
    # s = s' ∧ s' = s - (q * s') ∧ t = t' ∧ t' = t - (q * t')
    t0 = And(
        Equals(atual['node'], BV(0,bits)),
        NotEquals(atual['r2'], BV(0,bits)),
        Equals(prox['node'], BV(0,bits)),
        Equals(prox['r1'], atual['r2']),
        Equals(prox['r2'], BVSub(atual['r1'], BVMul(BVUDiv(atual['r1'], atual['r2']), atual['r2']))),
        Equals(prox['s1'], atual['s2']),
        Equals(prox['s2'], BVSub(atual['s1'], BVNeg(BVMul(BVUDiv(atual['r1'], atual['r2']), atual['s2'])))),
        Equals(prox['t1'], atual['t2']),
        Equals(prox['t2'], BVSub(atual['t1'], BVNeg(BVMul(BVUDiv(atual['r1'], atual['r2']), atual['t2']))))
    )
    
    # node = 0 ∧ next = 2 ∧ (r = 0 or overflow) ∧ r = r ∧ r' = r' ∧ s = s ∧ s' = s' ∧ t = t ∧ t' = t'
    t1 = And(
        Equals(atual['node'], BV(0,bits)),
        Equals(prox['node'], BV(2,bits)),
        Or(Equals(atual['r1'], BV(0,bits)), BVUGT(atual['s1'], m), BVUGT(atual['s2'], m),
           BVUGT(atual['t1'], m), BVUGT(atual['t2'], m)),
        Equals(prox['r1'], atual['r1']),
        Equals(prox['r2'], atual['r2']),
        Equals(prox['s1'], atual['s1']),
        Equals(prox['s2'], atual['s2']),
        Equals(prox['t1'], atual['t1']),
        Equals(prox['t2'], atual['t2'])
    )
    
    # node = 0 ∧ next = 1 ∧ r' = 0 ∧ r = r ∧ r' = r' ∧ s = s ∧
    # s' = s' ∧ t = t ∧ t' = t'
    t2 = And(
        Equals(atual['node'], BV(0,bits)),
        Equals(prox['node'], BV(1,bits)),
        Equals(atual['r2'], BV(0,bits)),
        Equals(prox['r1'], atual['r1']),
        Equals(prox['r2'], atual['r2']),
        Equals(prox['s1'], atual['s1']),
        Equals(prox['s2'], atual['s2']),
        Equals(prox['t1'], atual['t1']),
        Equals(prox['t2'], atual['t2'])
    )
    
    # node = 1 ∧ next = 1 ∧ r = r ∧ r' = r' ∧ s = s ∧ s' = s' ∧ t = t ∧ t' = t'
    t3 = And(
        Equals(atual['node'], BV(1,bits)),
        Equals(prox['node'], BV(1,bits)),
        Equals(prox['r1'], atual['r1']),
        Equals(prox['r2'], atual['r2']),
        Equals(prox['s1'], atual['s1']),
        Equals(prox['s2'], atual['s2']),
        Equals(prox['t1'], atual['t1']),
        Equals(prox['t2'], atual['t2'])
    )
    
    # node = 2 ∧ next = 2 ∧ r = r ∧ r' = r' ∧ s = s ∧ s' = s' ∧ t = t ∧ t' = t'
    t4 = And(
        Equals(atual['node'], BV(2,bits)),
        Equals(prox['node'], BV(2,bits)),
        Equals(prox['r1'], atual['r1']),
        Equals(prox['r2'], atual['r2']),
        Equals(prox['s1'], atual['s1']),
        Equals(prox['s2'], atual['s2']),
        Equals(prox['t1'], atual['t1']),
        Equals(prox['t2'], atual['t2'])
    )
    
    return Or(t0, t1, t2, t3, t4)

Função que gera um possível traço de execução do programa para k passos.

In [5]:
def exec_EXA(k):
    
    with Solver(name="z3") as s:
        #declara os estados
        trace = [declare(i, bits, 'Z') for i in range(k+1)]
        #adiciona restrição inicial
        s.add_assertion(init(trace[0], bits))

        #adiciona as transições ao solver
        for i in range(k):
            s.add_assertion(trans(trace[i], trace[i+1]))

        #executa o automato para os k passos
        if s.solve():
            for i in range(k):
                print("Passo", i)
                for v in trace[i]:
                    print(v, "=", s.get_value(trace[i][v]))
                print("-----------")


In [6]:
exec_EXA(10)

Passo 0
node = 0_9
r1 = 352_9
r2 = 88_9
s1 = 1_9
s2 = 0_9
t1 = 0_9
t2 = 1_9
-----------
Passo 1
node = 0_9
r1 = 88_9
r2 = 0_9
s1 = 0_9
s2 = 1_9
t1 = 1_9
t2 = 4_9
-----------
Passo 2
node = 1_9
r1 = 88_9
r2 = 0_9
s1 = 0_9
s2 = 1_9
t1 = 1_9
t2 = 4_9
-----------
Passo 3
node = 1_9
r1 = 88_9
r2 = 0_9
s1 = 0_9
s2 = 1_9
t1 = 1_9
t2 = 4_9
-----------
Passo 4
node = 1_9
r1 = 88_9
r2 = 0_9
s1 = 0_9
s2 = 1_9
t1 = 1_9
t2 = 4_9
-----------
Passo 5
node = 1_9
r1 = 88_9
r2 = 0_9
s1 = 0_9
s2 = 1_9
t1 = 1_9
t2 = 4_9
-----------
Passo 6
node = 1_9
r1 = 88_9
r2 = 0_9
s1 = 0_9
s2 = 1_9
t1 = 1_9
t2 = 4_9
-----------
Passo 7
node = 1_9
r1 = 88_9
r2 = 0_9
s1 = 0_9
s2 = 1_9
t1 = 1_9
t2 = 4_9
-----------
Passo 8
node = 1_9
r1 = 88_9
r2 = 0_9
s1 = 0_9
s2 = 1_9
t1 = 1_9
t2 = 4_9
-----------
Passo 9
node = 1_9
r1 = 88_9
r2 = 0_9
s1 = 0_9
s2 = 1_9
t1 = 1_9
t2 = 4_9
-----------


##### b)

In [7]:
#verifica se é um estado de erro
def error(atual):
    overflow = 2**bits - 1
    m = BV(overflow, bits)
    return Or(Equals(atual['r1'], BV(0,9)), BVUGT(atual['s1'], m), BVUGT(atual['s2'], m),
           BVUGT(atual['t1'], m), BVUGT(atual['t2'], m))

#verifica se são estados iguais
def same(state1,state2):
    return And([Equals(state1[x],state2[x]) for x in state1])

def baseName(s):
    return ''.join(list(itertools.takewhile(lambda x: x!='!', s)))

def rename(form,state):
    vs = get_free_variables(form)
    pairs = [ (x,state[baseName(x.symbol_name())]) for x in vs ]
    return form.substitute(dict(pairs))

#inverte função de transição
def invert(trans):
    return (lambda curr, prox: trans(prox, curr))

In [8]:
def model_checking(N, M):
    with Solver(name="z3") as solver:
        
        # Criar todos os estados que poderão vir a ser necessários.
        X = [declare(i, bits, 'X') for i in range(N+1)]
        Y = [declare(i, bits, 'Y') for i in range(M+1)]
        transt = invert(trans)
        
        # Estabelecer a ordem pela qual os pares (n,m) vão surgir. Por exemplo:
        order = sorted([(a,b) for a in range(1,N+1) for b in range(1,M+1)],key=lambda tup:tup[0]+tup[1])
        
        # implícito na ordem de 'order' e nas definições de Rn, Um.
        for (n,m) in order:
            
            I = init(X[0], bits)
            Tn = And([trans(X[i], X[i+1]) for i in range(n)])
            Rn = And(I, Tn)
            
            E = error(Y[0])
            Bm = And([transt(Y[i], Y[i+1]) for i in range(m)])
            Um = And(E, Bm)
            
            Vnm = And(Rn, same(X[n], Y[m]), Um)
            if solver.solve([Vnm]):
                print("> O sistema é inseguro.")
                return
            else:
                
                A = And(Rn, same(X[n], Y[m]))
                B = Um
                C = binary_interpolant(A, B)
                
                # Salvaguardar cálculo bem-sucedido do interpolante.
                if C is None:
                    print("> O interpolante é None.")
                    break
                
                
                C0 = rename(C, X[0])
                T = trans(X[0], X[1])
                C1 = rename(C, X[1])

                if not solver.solve([C0, T, Not(C1)]):
                    # C é invariante de T.
                    print("> O sistema é seguro.")
                    return 
                else:
                
                    S = rename(C, X[n])
                    while True:
                        
                        T = trans(X[n], Y[m])
                        A = And(S, T)
                        if solver.solve([A, Um]):
                            print("> Não foi encontrado majorante.")
                            break 
                        else:
                            
                            C = binary_interpolant(A, Um)
                            Cn = rename(C, X[n])
                            if not solver.solve([Cn, Not(S)]):
                            
                                # C(Xn) -> S é tautologia.
                                print("> O sistema é seguro.")
                                return
                            else:
                                
                                # C(Xn) -> S não é tautologia.
                                S = Or(S, Cn)
                                
    print("> Não foi provada a segurança ou insegurança do sistema.")
                            

model_checking(50, 50) 

> O sistema é seguro.
